# Analyse pyBadlands stratigraphic output

If the stratigraphic structure is turned on in the XmL input file, **pyBadlands** produces sedimentary layers Hdf5 files. The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we show how we can visualise quickly the structure of the stratigraphic layer in an IPython notebook.

In [ ]:
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters
from scipy.interpolate import RectBivariateSpline
from scipy.ndimage.filters import gaussian_filter

import pybadlands_companion.stratalAnalyse_DT as strata

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Loading stratigraphic file

First we need to load the stratigraphic file. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.region`ID`.time`T`.p`X`.hdf5`

with ID the region index, T the display time index and X the number of the partition (used in the parallel version). In cases where you ran your simulation in parallel you will also need to give the number of CPUs used (_cpus_).

To load a file you will need to give the folder name and the number of processors used in your simulation.

For more information regarding the function uncomment the following line.

In [ ]:
# help(strata.stratalSection.__init__)

In [ ]:
folder = '/workspace/volume/LEM/DynTopo/outputX/h5/'
strat = strata.stratalSection(folder)

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention).

**Note**

This number is not always the number of sedimentary layers for this particular time step as you could have chosen in the input file to have more than 1 sedimentary layer recorded by output interval!

In [ ]:
# help(strat.loadStratigraphy)

In [ ]:
strat.loadStratigraphy(100) 

# Building a cross-section

We then slice the stratigraphic mesh to visualise the sedimentary architecture along a given cross-section.

To create the cross-section you will need to provide the position of the segment in the simulation space _(xo,yo)_ and _(xm,ym)_, a gaussian filter value for smoothing (_gfilt_ a value of 0 can be used for non-smoothing) and the resolution of the cross-section (based on a number of points: _nbpts_). 

In [ ]:
# Build a cross-section along X axis
x1 = 0.
x2 = 300000.
y1 = 100000
y2 = 100000

# Interpolation parameters
nbpts = 301
gfilt = 1

In [ ]:
# help(strat.buildSection)

In [ ]:
strat.buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

# Stratal stacking pattern

We use **plotly** to visualise the vertival cross-section of stratal stacking apttern.

In [ ]:
#help(strata.viewSection)

In [ ]:
strata.viewSection(width = 800, height = 500, cs = strat, 
            dnlay = 2, rangeX=[220000, 300000], rangeY=[-1050,-50],
            linesize = 0.5, title='Stratal stacking pattern coloured by time')

# Interpretation of Systems Tracts

New analytical methods are proposed in recent years on the interpretation of depositional systems. Here, we apply three methods, including **(i) the original method** in which the definitation of systems tracts is based on relative sea-level change; **(ii) the shoreline trajectory analysis** which classifies different trajectory classes according to the migration of shoreline; **(iii) the accommodation sucession method** that focuses on the objective observation of depositional trends and then defines different sequence sets reponding to the competing ration between the change of accommodation and sediment supply.

## (1) Based on relative sea-level

There are several models of systems tracts within depositional sequences, here we use the most simple one called the four systems tract model (figure is from [Helland-Hansen & Hampson (2009)](http://onlinelibrary.wiley.com/wol1/doi/10.1111/j.1365-2117.2009.00425.x/full)): <img src="images/SystemsTract_RSL.png" alt="Systems tract defined based on relative sea-level"  width="300" height="300" border="10" />
- highstand systems tract **HST** 
- falling-stage systems tract **FSST**
- lowstand systems tract **LST**
- transgressive systems tract **TST**

For each of these system tracts we define a given color. We use the 'colorlover' library [link](http://moderndata.plot.ly/color-scales-in-ipython-notebook/)

We first visualize the sea-level surve, and then extract the time points that bound different systems tracts, as showed in the above figure.

In [ ]:
time, Sealevel = strata.readSea('/workspace/volume/LEM/DynTopo/data/sealevel.csv')
timeStep = 1e5
# Plot sea-level
strata.viewData(x0 = time/timeStep, y0 = Sealevel, width = 600, height = 400, linesize = 3, color = '#6666FF')

In [ ]:
# Define different colors for different systems tracts
cHST = 'rgba(213,139,214,0.8)' 
cFSST = 'rgba(215,171,117,0.8)' 
cLST = 'rgba(39,123,70,0.8)' 
cTST = 'rgba(82,128,233,0.8)' 
# Specify the extent of every systems tract according to the figure above
HST1 = np.array([0,11],dtype=int)
FSST1 = np.array([11,26],dtype=int)
LST1 = np.array([26,30],dtype=int)
TST1 = np.array([30,36],dtype=int)
HST2 = np.array([36,42],dtype=int)
FSST2 = np.array([42,69],dtype=int)
LST2 = np.array([69,78],dtype=int)
TST2 = np.array([78,88],dtype=int)
HST3 = np.array([88,100],dtype=int)
# Build the color list
STcolors = []
for k in range(HST1[0],HST1[1]):
    STcolors.append(cHST)
for k in range(FSST1[0],FSST1[1]):
    STcolors.append(cFSST)
for k in range(LST1[0],LST1[1]):
    STcolors.append(cLST)
for k in range(TST1[0],TST1[1]):
    STcolors.append(cTST)
for k in range(HST2[0],HST2[1]):
    STcolors.append(cHST)
for k in range(FSST2[0],FSST2[1]):
    STcolors.append(cFSST)
for k in range(LST2[0],LST2[1]):
    STcolors.append(cLST)
for k in range(TST2[0],TST2[1]):
    STcolors.append(cTST)
for k in range(HST3[0],HST3[1]):
    STcolors.append(cHST)

In [ ]:
# help(strata.viewSectionST)

In [ ]:
strata.viewSectionST(width = 800, height = 500, cs=strat, colors=STcolors,
                   dnlay = 2, rangeX=[220000, 300000], rangeY=[-1050,-50], 
                   linesize=0.5, title='Systems tract interpreted based on relative sea-level (RSL)')

Before doing the interpretation using another two methods, we need to calculate the shoreline positions (shoreline trajectory), the accommodation change ($\delta A$) and sedimentation change ($\delta S$) at shoreline position through time. The endpoint of every deposited layer is also calculated to plot the Wheeler Diagram.

In order to calculate the shoreline trajectory, $\delta A$ and $\delta S$ at the shoreline, we need the information of the sediemntary layers when they deposited. In this example, totally 100 outputs are obtained. We use a loop to read the output every two timesetps. 

In [ ]:
nbout = 51  # number of output file
folder = '/workspace/volume/LEM/DynTopo/outputX/h5/'

The `sed.time`T`.p`X`.hdf5` file starts from time1. Therefore, the first file we load is T=1, then followed by T=2, 4, 6, 8, ..., 100

In [ ]:
strat_all = [strata.stratalSection(folder)]
strat_all[0].loadStratigraphy(1)
for i in range(1,nbout):
    strat = strata.stratalSection(folder)
    strat_all.append(strat)
    strat_all[i].loadStratigraphy(i*2)

# Building cross-sections

We then slice the stratigraphic mesh to visualise the sedimentary architecture along a given cross-section.

To create the cross-section you will need to provide the position of the segment in the simulation space _(xo,yo)_ and _(xm,ym)_, a gaussian filter value for smoothing (_gfilt_ a value of 0 can be used for non-smoothing) and the resolution of the cross-section (based on a number of points: _nbpts_). 

In [ ]:
# Build a cross-section along X axis
x1 = 0.
x2 = 300000.
y1 = 100000
y2 = 100000

# Interpolation parameters
nbpts = 301
gfilt = 1

In [ ]:
# For multi-outputs
for i in range(0,nbout):
    strat_all[i].buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

In [ ]:
# Read sea-level file
time, sealevel = strata.readSea('/workspace/volume/LEM/DynTopo/data/sealevel.csv')
# Extract the value of sea-level at each extracted stratigraphic layer
dt = 2 # the number of timesteps to read the sed.region'ID'.time'T'.p'X'.hdf5 file
Time = np.array(time[::dt])
Sealevel = np.array(sealevel[::dt])
# Plot sea-level
strata.viewData(x0 = Time, y0 = Sealevel, width = 600, height = 400, linesize = 3, color = '#6666FF')

In [ ]:
def buildShoreline(cs = None, cs_b = None, sealevel = None, sealevel_b = None):
    shoreID = np.amax(np.where(cs.secDep[cs.nz-1]>=sealevel)[0])
    shoreElev = cs.secDep[cs.nz-1][shoreID]
    shoreID_b = np.amax(np.where(cs_b.secDep[cs_b.nz-1]>=sealevel_b)[0])
    accom = sealevel - cs.secDep[cs_b.nz-1][shoreID_b]
    sed = cs.secDep[cs.nz-1][shoreID_b] - cs.secDep[cs_b.nz-1][shoreID_b]
    depoend = np.amax(np.where(cs.secTh[cs.nz-1][shoreID:len(cs.secTh[0])]>0.001)[0]) + shoreID
    sedflux = 0.
    for i in range(cs_b.nz-1,cs.nz):
        sedflux = sedflux + sum(cs.secTh[i][shoreID:,])
            
    return shoreID, shoreElev, accom, sed, depoend, sedflux

In [ ]:
# Calculate the shoreline trajectory
shoreID = np.zeros(nbout)
shoreElev = np.zeros(nbout)
accom_shore = np.zeros(nbout)
sed_shore = np.zeros(nbout)
depoend = np.zeros(nbout)
sedflux = np.zeros(nbout)
# 
shoreID[0], shoreElev[0], accom_shore[0], sed_shore[0], depoend[0], sedflux[0] = buildShoreline(cs = strat_all[0], cs_b = strat_all[0], sealevel = Sealevel[0], sealevel_b = Sealevel[0])
for i in range(1,nbout):
    shoreID[i], shoreElev[i], accom_shore[i], sed_shore[i], depoend[i], sedflux[i] = buildShoreline(cs = strat_all[i], cs_b = strat_all[i-1], sealevel = Sealevel[i], sealevel_b = Sealevel[i-1])

In [ ]:
# Gaussian smooth
shoreID_gs = filters.gaussian_filter1d(shoreID,sigma=1)
shoreElev_gs = filters.gaussian_filter1d(shoreElev,sigma=1)
accom_gs = filters.gaussian_filter1d(accom_shore,sigma=1)
sed_gs = filters.gaussian_filter1d(sed_shore,sigma=1)
depoend_gs = filters.gaussian_filter1d(depoend,sigma=1)
sedflux_gs = filters.gaussian_filter1d(sedflux,sigma=1)

## (2) Based on shoreline trajectory

Here is another way to interpret the depositional units based on the shifts of shoreline (from [Helland-Hansen & Hampson (2009)](http://onlinelibrary.wiley.com/wol1/doi/10.1111/j.1365-2117.2009.00425.x/full)):
<img src="images/SystemsTract_TC.png" alt="Systems tract defined based on shoreline trajectory"  width="400" height="400" border="10" />

- Transgressive trajectory class ** TTC **, when shoreline landward shift
- Descending regressive trajectory class ** DRTC **, when shoreline basinward shift accompanied by aggradation
- Ascending regressive trajectory class ** ARTC **, when shoreline basinward shift accompanied by degradation
- Stationary trajectory class **STC**, when the shoreline stands still

In [ ]:
# Visualize the shoreline trajectory
strata.viewData(x0 = Time/timeStep, y0 = shoreID, width = 600, height = 400, linesize = 3, color = '#6666FF')
# Smoothed shoreline trajectory
strata.viewData(x0 = Time/timeStep, y0 = shoreID_gs, width = 600, height = 400, linesize = 3, color = '#6666FF')

In [ ]:
# Define different colors for different trajectory classes
cTC = 'rgba(56,110,164,0.8)' 
cDRC = 'rgba(60,165,67,0.8)'  
cSTC = 'rgba(246,146,80,0.8)'
# Specify the extent of every trajectory class according to the figure above
TC1 = np.array([0,10],dtype=int)  # when shoreline landward shift
DRC1 = np.array([10,26],dtype=int)  # when shoreline seaward shift
TC2 = np.array([26,42],dtype=int)
DRC2 = np.array([42,60],dtype=int)
STC1 = np.array([60,76],dtype=int)
TC3 = np.array([76,100],dtype=int)
# Build the color list
STcolors_ST = []
for k in range(TC1[0],TC1[1]):
    STcolors_ST.append(cTC)
for k in range(DRC1[0],DRC1[1]):
    STcolors_ST.append(cDRC)
for k in range(TC2[0],TC2[1]):
    STcolors_ST.append(cTC)
for k in range(DRC2[0],DRC2[1]):
    STcolors_ST.append(cDRC)
for k in range(STC1[0],STC1[1]):
    STcolors_ST.append(cSTC)
for k in range(TC3[0],TC3[1]):
    STcolors_ST.append(cTC)

In [ ]:
strata.viewSectionST(width = 800, height = 500, cs = strat, colors = STcolors_ST, 
                     dnlay = 2, rangeX=[220000, 300000], rangeY=[-1050,-50], 
                     linesize = 0.5, title = 'Systems tract interpreted based on shoreline trajectory (ST)')

## (3) Based on the change of accommodation & sedimentation

This accommodation succession method is referred to [Neal & Abreu (2009)]((https://www.researchgate.net/publication/249521744_Sequence_stratigraphy_hierarchy_and_the_accommodation_succession_method)), which is based on the relationship between the rate of changes in accommodation and sedimentation. 
<img src="images/SystemsTract_AS.png" alt="Systems tract defined based on shoreline trajectory"  width="600" height="400" border="10" />

- Retrogradation sequence set ** RSS **
- Aggradation to Progradation (maybe) to Degradation sequence set ** APDSS **
- Pragradation to Aggradation sequence set ** PASS **

In [ ]:
# Visualize the difference between the change of accommodation (A) and that of sedimentation (S): A-S
strata.viewData(x0 = Time/timeStep, y0 = accom_gs-sed_gs, width = 600, height = 400, linesize = 3, color = '#6666FF')

In [ ]:
# Define different colors for different sequence sets
cR = 'rgba(51,79,217,0.8)' 
cPA = 'rgba(252,149,7,0.8)' 
cAPD= 'rgba(15,112,2,0.8)'
# Specify the extent of every sequence set according to the figure above
R1 = np.array([0,12],dtype=int)  # when A-S>0
APD1 = np.array([12,18],dtype=int)  # when A-S<0 but increasing
PA1 = np.array([18,28],dtype=int)  # when A-S <0 but decreasing
R2 = np.array([28,42],dtype=int)
APD2 = np.array([42,62],dtype=int)
PA2 = np.array([62,76],dtype=int)
R3 = np.array([76,94],dtype=int)
APD3 = np.array([94,100],dtype=int)
# Build the color list
STcolors_AS = []
for k in range(R1[0],R1[1]):
    STcolors_AS.append(cR)
for k in range(APD1[0],APD1[1]):
    STcolors_AS.append(cAPD)
for k in range(PA1[0],PA1[1]):
    STcolors_AS.append(cPA)
for k in range(R2[0],R2[1]):
    STcolors_AS.append(cR)
for k in range(APD2[0],APD2[1]):
    STcolors_AS.append(cAPD)
for k in range(PA2[0],PA2[1]):
    STcolors_AS.append(cPA)
for k in range(R3[0],R3[1]):
    STcolors_AS.append(cR)
for k in range(APD3[0],APD3[1]):
    STcolors_AS.append(cAPD)

In [ ]:
strata.viewSectionST(width = 800, height = 500, cs = strat, colors = STcolors_AS, 
                     dnlay = 2, rangeX=[220000, 300000], rangeY=[-1050,-50], 
                     linesize = 0.5, title = 'Systems tract interpreted based on change of accommodation and sedimentation (AS)')

# Depositional environments

We define different depositional environments (depositional facies) based on water depth when the strata are deposited. For example,

- **alluvial plain**:   30m above sea level
- **shoreface**:        0 - 50m below sea level
- **shallow marine**:   50 - 200m below sea level
- **slope**:            200 - 400m below sea level
- **deep marine**:      400 - 600m below sea level
- **ocean basin**:      > 600m below sea level

In [ ]:
# help(strata.depthID)

In [ ]:
envIDs = [-30, 0, 50, 200, 400, 600]  # water depth relative to sea level, positive means below sea level
waterIDs = np.zeros((len(envIDs), nbout))
for i in range(0,nbout):
    waterIDs[:,i] = strata.depthID(cs = strat_all[i], sealevel = Sealevel[i], envIDs = envIDs)

In [ ]:
fig = plt.figure(figsize = (8,4))
plt.rc("font", size=14)
# 
ax = fig.add_subplot(111)
layID = []
p = 0
color = ['limegreen','darkkhaki','sandybrown','khaki','c']
xi00 = strat.dist/1000  # change the unit to be be km
for i in range(0,strat.nz+1,dn):
    if i == strat.nz:
        i = strat.nz-1
    layID.append(i)
    if len(layID) > 1:
        for j in range(0,int(np.amax(xi00))):
            if (j<waterIDs[0][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color='limegreen')
            elif (j<waterIDs[1][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[0])
            elif (j<waterIDs[2][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[1])
            elif (j<waterIDs[3][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[2])
            elif (j<waterIDs[4][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[3])
            elif (j<waterIDs[5][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[4])
            else:
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color='teal')
                plt.fill_between(xi00, strat.secDep[layID[p]], 0, color='white')
    p=p+1
    plt.plot(xi00,strat.secDep[i],'-',color='k',linewidth=0.3)
#       
plt.plot(xi00,strat.secDep[strat.nz-1],'-',color='k',linewidth=0.7)
plt.plot(xi00,strat.secDep[0],'-',color='k',linewidth=0.7)
plt.ylabel('Elevation (m)',fontsize=14)
plt.xlabel('Distance (km)',fontsize=14)
plt.xlim( 220, 300 ) 
plt.ylim( -1050, -50 )
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
plt.show()
#fig.savefig("/workspace/volume/figure/stacking.png", dpi=400)

# Wheeler Diagram

Wheeler diagram (or chronostratigraphic chart) is a powerful tool to document unconformities between sequences, and to understand the stacking patterns of sedimentary successions and their relationship to sea level. It displays the horizontal distribution of contemporaneous sedimentary layer sequences, as well as hiatuses in sedimentation. 

In addition to present the horizontal distribution of depositional environments, we also display the shoreline trajectory.

In [ ]:
# help(strata.depthIDs)

In [ ]:
envi = np.append(waterIDs, [depoend], axis=0)

In [ ]:
fig = plt.figure(figsize = (8,4))
plt.rc("font", size=14)
# 
# xmin = np.amin(envi[0,:])-10
# xmax = np.amax(envi[envi.shape[0]-1,:])+10
# 
Time_new = [x/1e6 for x in Time]  # change the unit to be Myr
color = ['limegreen','darkkhaki','sandybrown','khaki','c', 'teal', 'white']
patch_handles = []
for i, d in enumerate(envi):
    patch_handles.append(plt.barh(Time_new, d, 
    color=color[i], align='edge', left=d, height=dn/10., edgecolor = "none"))
for j in range(0,nbout,1):
    plt.axhline(Time_new[j], color='k', linewidth=0.5)
plt.plot(shoreID, Time_new,'ko',markersize=3)
plt.xlim([200, 300])
plt.ylim([-0.2,10.2])
# plt.grid(True)
plt.yticks([0, 5, 10])
plt.xlabel('Distance (km)',fontsize=14)
plt.xticks(fontsize=14)
plt.ylabel('Time (Myr)',fontsize=14)
plt.yticks(fontsize=14)
plt.show()
#fig.savefig("/workspace/volume/figure/wheelerDiag.png", dpi=400)

# Vertical stacking pattern

Vertical stacking patterns (synthetic wells) could also be extracted at different positions to show the thickness of parasequences and the trend of depositional units through time.

To extract the vertical staking, first we need to calculate the stacked strata thickness at each position, then color the strata according to the water depth when they are deposited.

In [ ]:
posit = [230, 240, 250, 260, 270]  # position of wells /km
envIDs_new = [-30, 0, 50, 200, 400, 600, 1200]
color_new = ['white','limegreen','darkkhaki','sandybrown','khaki','c', 'teal']

In [ ]:
def vStack(cs = None, posit = None, envIDs = None, color = None, nbout = None, dn = None):
    color_fill = []
    for i in range(len(posit)):
        for j in range(0,cs.nz,dn):
            if ((cs.secElev[j][posit[i]]) > (- envIDs[0])):
                color_fill.append(color[0])
            elif (cs.secElev[j][posit[i]]) > (- envIDs[1]):
                color_fill.append(color[1])
            elif (cs.secElev[j][posit[i]]) > (- envIDs[2]):
                color_fill.append(color[2])
            elif (cs.secElev[j][posit[i]]) > (- envIDs[3]):
                color_fill.append(color[3])
            elif (cs.secElev[j][posit[i]]) > (- envIDs[4]):
                color_fill.append(color[4])
            elif (cs.secElev[j][posit[i]]) > (- envIDs[5]):
                color_fill.append(color[5])
            else:
                color_fill.append(color[6])          
    #
    layth = []
    for m in range(len(posit)):
        nz = cs.nz-1
        layth.append(cs.secDep[nz][posit[m]])
        for n in range(1,nbout-1):
            layth.append(-sum(cs.secTh[(nz-n*dn):(nz-(n-1)*dn)])[posit[m]])
    #
    colorFill = np.reshape(color_fill, (len(posit), nbout-1))
    layTh = np.reshape(layth, (len(posit), nbout-1))
    
    return colorFill, layTh

In [ ]:
colorFill, layTh = vStack(cs = strat, posit = posit, envIDs = envIDs_new, color = color_new, nbout = nbout, dn = dn)

In [ ]:
import matplotlib as mpl
fig = plt.figure(figsize = (4,5))
plt.rc("font", size=10)
# 
ax = fig.add_axes([0.2,0.06,0.82,0.91])
data = layTh
bottom1 = np.cumsum(data[0], axis=0)
colors_1 = np.fliplr([colorFill[0]])[0]
plt.bar(0, data[0][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[0].shape[0]):
    plt.bar(0, data[0][j], color=colors_1[j], bottom=bottom1[j-1])
# 
bottom2 = np.cumsum(data[1], axis=0)
colors_2 = np.fliplr([colorFill[1]])[0]
plt.bar(2, data[1][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[1].shape[0]):
    plt.bar(2, data[1][j], color=colors_2[j], bottom=bottom2[j-1])
# 
bottom3 = np.cumsum(data[2], axis=0)
colors_3 = np.fliplr([colorFill[2]])[0]
plt.bar(4, data[2][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[2].shape[0]):
    plt.bar(4, data[2][j], color=colors_3[j], bottom=bottom3[j-1])
#
bottom4 = np.cumsum(data[3], axis=0)
colors_4 = np.fliplr([colorFill[3]])[0]
plt.bar(6, data[3][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[3].shape[0]):
    plt.bar(6, data[3][j], color=colors_4[j], bottom=bottom4[j-1])
#
bottom5 = np.cumsum(data[4], axis=0)
colors_5 = np.fliplr([colorFill[4]])[0]
plt.bar(8, data[4][0], color = 'w', edgecolor='lightgrey', hatch = '/')
for j in range(1, data[4].shape[0]):
    plt.bar(8, data[4][j], color=colors_5[j], bottom=bottom5[j-1])
#
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.axes.get_xaxis().set_visible(False)
ax.tick_params(axis='both', labelsize=10)
ax.yaxis.set_ticks_position('left')
plt.xlim(0,10)
plt.ylim(-800,0)
plt.ylabel('Elevation (m)',fontsize=12)
plt.yticks(fontsize=12)
plt.show()
#fig.savefig("/workspace/volume/figure/well.png", dpi=400)